In [ ]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
# Config LLM
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model='gpt-4o'
)

In [15]:
# Config embeddings
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(
    model='text-embedding-3-large'
)

In [8]:
urls=[
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
]

In [11]:
from langchain_community.document_loaders import WebBaseLoader
docs_arr = [WebBaseLoader(url).load() for url in urls]
docs = [i for url_docs in docs_arr for i in url_docs]

In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 100,
    chunk_overlap = 25
)

In [20]:
docs_splits = text_splitter.split_documents(docs)

In [22]:
from langchain_community.vectorstores import Chroma
vector_db = Chroma.from_documents(
    documents=docs_splits,
    embedding=embeddings
)

In [24]:
retriever=vector_db.as_retriever()

In [57]:
from langchain.tools.retriever import create_retriever_tool
retriever_node=create_retriever_tool(
    retriever,
    "retriever_blog_post",
    "Search and return information about Lilian Weng blog posts on LLM agents, prompt engineering, and adversarial attacks on LLMs.You are a specialized assistant. Use the 'retriever_tool' **only** when the query explicitly relates to LangChain blog data. For all other queries, respond directly without using any tool. For simple queries like 'hi', 'hello', or 'how are you', provide a normal response.",
    )

In [58]:
from langgraph.prebuilt import ToolNode
tools = [retriever_node]
retriever_node = ToolNode(tools)

In [30]:
tools

[Tool(name='retriever_blog_post', description="Search and return information about Lilian Weng blog posts on LLM agents, prompt engineering, and adversarial attacks on LLMs.You are a specialized assistant. Use the 'retriever_tool' **only** when the query explicitly relates to LangChain blog data. For all other queries, respond directly without using any tool. For simple queries like 'hi', 'hello', or 'how are you', provide a normal response.", args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x126cf7ce0>, retriever=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x117299160>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant

In [60]:
from langchain import hub
hub.pull("rlm/rag-prompt").pretty_print()

================================ Human Message =================================

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


In [54]:
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import BaseMessage
import operator


In [55]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [66]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain_core.prompts import PromptTemplate

In [64]:
class grade(BaseModel):
    binary_score:str=Field(description="Relvance score 'yes' or 'no'")

In [67]:
# Grade Node
def grade_documents(state:AgentState)->Literal["Output Generator", "Query Rewriter"]:
    print("----CALLING GRADE FOR CHECKING RELEVANCY----")
    llm_with_structure_op=llm.with_structured_output(grade)
    
    prompt=PromptTemplate(
        template="""You are a grader deciding if a document is relevant to a user’s question.
                    Here is the document: {context}
                    Here is the user’s question: {question}
                    If the document talks about or contains information related to the user’s question, mark it as relevant. 
                    Give a 'yes' or 'no' answer to show if the document is relevant to the question.""",
                    input_variables=["context", "question"]
                    )
     
    chain=prompt|llm_with_structure_op
     
     
    message=state['messages']
    
    last_message = message[-1]
    
    question = message[0].content
    
    docs = last_message.content
    
    scored_result=chain.invoke({"question": question, "context": docs})
    
    score=scored_result.binary_score
     
    if score=="yes":
        print("----DECISION: DOCS ARE RELEVANT----")
        return "generator"
    else:
        print("----DECISION: DOCS ARE NOT RELEVANT----")
        return "rewriter"

In [68]:
def generate(state:AgentState):
    print("----RAG OUTPUT GENERATE----")
    
    message=state["messages"]
    question=message[0].content
    
    last_message = message[-1]
    docs = last_message.content
    
    prompt=hub.pull("rlm/rag-prompt")
    
    rag_chain=prompt | llm
    
    response=rag_chain.invoke({"context": docs, "question": question})
    
    print(f"this is my response:{response}")
    
    return {"messages": [response]}

In [69]:
from langchain_core.messages import HumanMessage
def rewrite(state:AgentState):
    print("----TRANSFORM QUERY----")
    message=state["messages"]
    
    question=message[0].content
    
    input= [HumanMessage(content=f"""Look at the input and try to reason about the underlying semantic intent or meaning. 
                    Here is the initial question: {question} 
                    Formulate an improved question: """)
       ]

    response=llm.invoke(input)
    
    return {"messages": [response]}

In [91]:
prompt = hub.pull("rlm/rag-prompt").pretty_print()


================================ Human Message =================================

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


In [87]:
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_tavily import TavilySearch

def tavily_search(state: AgentState):
    os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
    tool = TavilySearch(
        max_results = 5,
        topic="general"
    )
    question = messages[0].content
    context = tool.invoke({"query":question})
    prompt = hub.pull("rlm/rag-prompt").pretty_print()
    messages = state["messages"]
    chain = prompt | llm
    chain.invoke({"query":question,"context":context})

    

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import tools_condition


In [98]:
workflow = StateGraph(AgentState)
workflow.add_node('Retriever Node',retriever_node)
workflow.add_node('Grade Node',grade_documents)
workflow.add_node('Re-write query', rewrite)
workflow.add_node('Web Search',tavily_search)
workflow.add_node('Generator',generate)

workflow.add_edge(START, 'Retriever Node')
workflow.add_edge('Retriever Node', 'Grade Node')
workflow.add_conditional_edges('Grade Node',
                               tools_condition,
                               {
                                   "generator": "Generator",
                                   "rewriter": "Re-write query"
                               }
                               
                               )

workflow.add_edge("Re-write query", 'Web Search')
workflow.add_edge('Web Search', END)

In [100]:
app = workflow.compile()

In [102]:
app.invoke({"messages": ["Who won the IPL 2025?"]})

ValueError: No AIMessage found in input